-----------------

# **说明**

### Baseline 方法为无监督方法，即不使用反向传播更新网络，直接使用test数据集推理，不使用train数据集进行监督学习训练，train用于评测无监督结果

### 简单思路：

### 本比赛主要有两个数据信息，图片信息和文本标题信息，你需要利用这两个信息，根据test所给的图像，检索出train数据集中与之相似的同款图片

### 对于图片信息，我们使用 Resnet18 或 Resnet50 进行特征提取

### 你若不知道resnet是什么可以看链接： 原始论文：https://arxiv.org/abs/1512.03385   人话：https://www.cnblogs.com/shine-lee/p/12363488.html

### 对于文本信息，我们使用 TF-IDF 信息进行特征提取


### 分别计算两个特征各自内部的相似度，最后得到综合结果


In [ ]:
import os, sys
sys.path = ['../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master', ] + sys.path

---------------------

# **第一步 导包**

In [ ]:
# 基本操作包
import numpy as np
import pandas as pd

import random
# 图片加载包
import cv2, matplotlib.pyplot as plt
from PIL import Image

# 进度条辅助包
from tqdm.notebook import tqdm

# 正则化和TF-IDF包，用于提取文本特征
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer

# 导入深度学习框架pytorch用于使用resnet网络提取图像特征
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

from efficientnet_pytorch import EfficientNet

# **第二步 固定随机数**

In [ ]:
# 随机性固定
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ.setdefault('DJANGO_SETTINGS_MODULE','first_project.settings')
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # 是否将卷积算子的计算实现固定。torch 的底层有不同的库来实现卷积算子
    torch.backends.cudnn.deterministic = True
    # 是否开启自动优化，选择最快的卷积计算方法
    torch.backends.cudnn.benchmark = True

In [ ]:
seed_torch(2021)

# **第三步 配置基本Config信息**

In [ ]:
# 设置数据集的路径，这样后续就可以直接快速使用了。
DATA_PATH = '../input/shopee-product-matching/'
# 批大小
BATCH_SIZE = 32
# 多线程
Num_workers = 2
# CNN特征相似度大于95%才列入相似
CNN_Confident = 0.95
# TF-IDF特征相似度大于80%才能列入相似
TFIDF_Confident = 0.80
# 是否用于提交，本地测试的时候请改为False，查看交叉验证成绩，提交时请改为True
IS_SUB = True
# IS_SUB = False

# **第四步 加载数据集**

In [ ]:
# 读取数据集
if IS_SUB:
    query = pd.read_csv(DATA_PATH + 'test.csv')
else:
    query = pd.read_csv(DATA_PATH + 'train.csv')

# 打印数据集信息
print('query shapes', query.shape)
query.head()

### 可以发现数据由5个属性构成，分别是如下意思

### posting_id 图片id号，用于标识图片；

### image 图片文件名，需要加上DATA_PATH才是真正的地址；

### image_phash 图片哈希，即图片指纹，用于快速鉴别重复图像；

### title 图片描述的商品名，文本信息

### label_group 相同数字代表同一类东西。（测试集没有，在无监督的方法中，我们不使用该属性，你可以思考监督方法怎么应用这一个属性）

### **为PyTorch框架构建读取数据集**

In [ ]:
# 继承pytorch的Dataset，重写方法，必须要__getitem__和__len__两个方法的重写。
class load_dataset_for_pyTorch(Dataset):
    def __init__(self, img_path, transform):
        # 图片地址
        if IS_SUB:
            self.img_path = DATA_PATH + 'test_images/' + img_path
        else:
            self.img_path = DATA_PATH + 'train_images/' + img_path
        # 图片变换
        self.transform = transform
        
    def __getitem__(self, index):
        # 读取图片
        img = Image.open(self.img_path[index]).convert('RGB')
        # 进行图片变换
        img = self.transform(img)
        return img
    
    def __len__(self):
        # 返回图片长度
        return len(self.img_path)

In [ ]:
image = load_dataset_for_pyTorch(query['image'].values,transforms.Compose([transforms.Resize((512, 512)),# 裁剪至512 * 512
                                                                           transforms.ToTensor(), # 转换成张量形式
                                                                           transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])) # 正则化
    
imageloader = torch.utils.data.DataLoader(image,batch_size=BATCH_SIZE, shuffle=False, num_workers=Num_workers) # 转换为pytorch的数据加载器

In [ ]:
# 测试是否正常生成：
testloader = torch.utils.data.DataLoader(image,batch_size=BATCH_SIZE, shuffle=False, num_workers=Num_workers) # 转换为pytorch的数据加载器
batch = next(iter(testloader))
print('正常读取图片！数据维度为：', batch.shape)

# **第五步 计算哈希值，用于快速筛查重复图片**

In [ ]:
# 将相同的哈希值聚集在一起
repeat = query.groupby('image_phash').posting_id.agg('unique').to_dict()
# 将聚集结果映射到每一个图片上
query['hash'] = query.image_phash.map(repeat)
# 查看结果
query.head()

# **第六步 利用Resnet18计算图片特征**

### **首先创建pytorch模型**

In [ ]:
# class Resnet18(nn.Module):
#     # 初始化
#     def __init__(self):
#         super(Resnet18, self).__init__()
#         # 快速加载pytorch提供好的resnet
#         model = models.resnet18(True)
#         # 对于图像检索任务，下采样通常使用最大下采样，因为需要找到突出特点
# #         model.avgpool = nn.AdaptiveMaxPool2d(output_size=(1, 1))
#         # 去除最后一层全连接分类层，只要倒数第二个特征图
#         model = nn.Sequential(*list(model.children())[:-1])
#         # 停用dropout，进入推理状态
#         model.eval()
#         self.model = model
        
#     # 前向推理
#     def forward(self, img):        
#         out = self.model(img)
#         return out
# class EfficientNet(nn.Module):
#     def __init__(self, backbone='ef', out_dim=6):
#         super(EfficientNet, self).__init__()
#         self.enet = EfficientNet.from_name('efficientnet-b1')
        
#         self.fc = nn.Linear(self.enet._fc.in_features, out_dim)
#         self.enet._fc = nn.Identity()
    
#     def forward(self, x):
#         x = self.enet(x)
#         x = self.fc(x)
#         return x

### **离线状态下拷贝预训练模型**

本来我想上传到数据集的，结果发现别人早就上传了，然后kaggle有查重机制，不可上传跟别人一样的文件，所以这里只能用别人的数据集包，右边Add data 搜索：Pretrained PyTorch models，第一个即可

In [ ]:
!mkdir -p /root/.cache/torch/hub/checkpoints/
!cp ../input/pretrained-pytorch-models/resnet18-5c106cde.pth /root/.cache/torch/hub/checkpoints/

### **声明使用模型 (这部分其实建议使用GPU加速，不然很慢，CPU：2小时推理 GPU：10分钟推理)**

In [ ]:
net = EfficientNet.from_name('efficientnet-b7')

 
# open_gpu = False
open_gpu = True

if open_gpu:
    net.to(torch.device('cuda'))

### **前向传播捕获特征**

In [ ]:
# 用于存储特征
image_features = []

with torch.no_grad():
    # 循环数据读取
    for data in tqdm(imageloader):
        if open_gpu:
            data = data.to(torch.device('cuda'))
        # 前向传播
        features = net.extract_features(data)
        # 将输出结果 [batch_size, 512, 1, 1] 变化成 [batch_size, 512]
        features = features.reshape(32, -1)
        
        # 若为gpu版本，训练好后提取为cpu版本
        if open_gpu:
            features = features.data.cpu().numpy()
        
        # 存储结果
        image_features.append(features)

# 展开维度
image_features = np.vstack(image_features)
# 正则化
image_features = normalize(image_features)
image_features=image_features.reshape(3,-1)
# 查看
image_features[:1].shape

print(image_features.shape)

### **计算相似度**

In [ ]:
# 存储结果
temp = []

# 分块计算，免得矩阵太大
CHUNK = 4096
times = len(image_features)//CHUNK

# 补全最后不刚好完整的块
if len(image_features) % CHUNK != 0:
    times += 1

# 计算相似度
for index in range( times ):
    # 每次运算的起点
    left = index * CHUNK
    # min的作用是防止超界
    right = min((index + 1) * CHUNK, len(image_features))
    
    print('chunk',left,'to',right)
    
    distances = np.dot(image_features[left:right,], image_features.T)
    print(image_features[left:right,].shape)    
    for k in range(right - left):
        # 取置信度大于CNN_Confident相似的，可以调这个上分
        IDX = np.where(distances[k,]>CNN_Confident)[0][:]
        # 将相似的图片加入到结果中
        
        o = query.iloc[IDX].posting_id.values
        temp.append(o)

query['cnn'] = temp

# **第七步 利用TF-IDF计算文本相似度**

### **搭建模型 前向传播 计算特征**

In [ ]:
# sklearn构建模型
model = TfidfVectorizer(stop_words=None, binary=True, max_features=10000)
# 前向传播
text_features = model.fit_transform(query.title).toarray()

### **计算相似度**

In [ ]:
# 存储结果
temp = []

# 分块计算，免得矩阵太大
CHUNK = 4096
times = len(text_features)//CHUNK

# 补全最后不刚好完整的块
if len(text_features) % CHUNK != 0:
    times += 1

# 计算相似度
for index in tqdm(range( times )):
    # 每次运算的起点
    left = index * CHUNK
    # min的作用是防止超界
    right = min((index + 1) * CHUNK, len(text_features))
    
    print('chunk',left,'to',right)
    
    distances = np.dot(text_features[left:right,], text_features.T)
    
    for k in range(right - left):
        # 取置信度大于CNN_Confident相似的，可以调这个上分
        IDX = np.where(distances[k,]>TFIDF_Confident)[0]
        # 将相似的图片加入到结果中
        o = query.iloc[IDX].posting_id.values
        temp.append(o)

del model, text_features
query['tfidf'] = temp

# **第八步 合成结果提交**

In [ ]:
# 合成函数用于提交【模板】
def combine_for_sub(row):
    # 结合三种答案的所有答案
    x = np.concatenate([row.hash, row.cnn, row.tfidf])
    # unique去重
    return ' '.join( np.unique(x) )



# 合成函数用于本地测试【模板】
def combine_for_cv(row):
    x = np.concatenate([row.hash, row.cnn, row.tfidf])
    return np.unique(x)

# 得到分数【通用模板】
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]))
        if len(row[col])==0:
            p = 0
        else:
            p = n/len(row[col])
        if len(row.target) == 0:
            r = 0
        else:
            r = n/len(row.target)
        return p, r, 2*n/(len(row.target)+len(row[col]))
    return f1score

In [ ]:
# 应用合成【模板】
query['matches'] = query.apply(combine_for_sub,axis=1)


# 保存结果【模板】
query[['posting_id','matches']].to_csv('submission.csv',index=False)
sub = pd.read_csv('submission.csv')
sub.head()

# **附录：若为本地，可根据label_group测试结果**

In [ ]:
if not IS_SUB:
    # 将相同group号聚集
    temp = query.groupby('label_group').posting_id.agg('unique').to_dict()
    # 映射到答案上
    query['target'] = query.label_group.map(temp)
    # 更改格式用于计算分数
    query['matches'] = query.apply(combine_for_cv,axis=1)
    # 计算F1分数
    query['score'] = query.apply(getMetric('matches'),axis=1)
    # 查看结果
    print('本地测试分数：')
    print('准确率P分数：',query['score'].apply(lambda x:x[0]).mean())
    print('召回率R分数：',query['score'].apply(lambda x:x[1]).mean())
    print('F1分数：',query['score'].apply(lambda x:x[2]).mean())

# **注：你还可以使用cudf, cuml, cupy代替numpy和pandas的运算，将运算迁移至cuda gpu上加速运算，这样TF-IDF可以使用更大的特征维度**